In [1]:
import time

import numpy as np
# import torch

import tools.rotations as rot
# import pytorch_kinematics as pk

from controller import locomotion
from control_interface_real_robot import real_robot
import mujoco
import cvxpy as cp
from control_interface_real_robot import q_sim2real

import copy
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'rclpy'

In [ ]:
import sys
sys.executable

In [ ]:
hand_real = real_robot(control_mode='position', legs=5)

In [ ]:
hand_real.update_data()
hand_real.q

In [2]:
# test pytorch kinematics
import copy
xml_path = 'descriptions/five_finger_hand_ssss.xml'
# chain = pk.build_chain_from_mjcf('descriptions/single_finger_ssss.xml', 'finger_1')
# print(chain)
var_num = 12
q0 = np.array([-1.62289571e-04, -3.50667052e-04,  7.98538815e-02,  9.99999738e-01,
       -4.46340938e-04,  5.67392530e-04, -4.89152197e-05, 
               -3.86962861e-05,  7.91144283e-01,  0.3,  0.3, 
              1,  7.93214611e-01,  0.3,  0.3,  
               0.00000000e+00, -1.47079633e+00, -8.00000000e-01, -8.00000000e-01, 
               -6.26414686e-06,  7.93034999e-01,  0.3,  0.3,
               -1.00000000e+00,  7.92384667e-01,  0.3,  0.3])
r = locomotion(xml_path, var_num=var_num, q0=q0)

print('locomotion freq', r.f)
# take a rest to let the acceleration to zero
t0 = time.time()
q0 = np.copy(r.q)
while 1:
    # r.step()
    # r.send_torque(r.C)
    r.joint_computed_torque_control(q0)
    if time.time() - t0 > 2:
        break
xh_local_init = copy.deepcopy(r.xh_local) 

Kinematic model has been loaded from kinematics/q2pose_5.txt
locomotion freq 1.0


In [ ]:
# q_cmd = np.zeros(12)
# q_cmd[0] = 0.5
# hand_real.move_to_joints(q_cmd)

# q_cmd_sim = np.zeros(20)
# q_cmd_sim[13] = 0 
# q_cmd_sim[17] = 0
# r.modify_joint(q_cmd_sim)

q_cmd = q_sim2real(q0)

hand_real.move_to_joints(q_cmd, sim2real=False)  # move to init pose
# hand_real.send_position(q_cmd)
# hand_real.move_to_joints_by_vel(q_cmd)


In [ ]:
# sin traj test
t0 = time.time()
q_cmd_0  = copy.deepcopy(q_cmd.flatten())
f = 1

q_record = []
q_record_ref = []
i = 1
while 1:
    hand_real.update_data()
    q = copy.deepcopy(q_cmd_0)
    t = time.time() - t0
    q[1] = q_cmd_0[i] + 0.4 * np.sin(2*np.pi*f*t) 
    dq = 0.4 * np.cos(2*np.pi*f*t) * 2*np.pi*f
    hand_real.send_position(q)
    q_record_ref.append(np.array([t, q[i], dq]))
    q_record.append(np.array([t, hand_real.q[i], hand_real.dq[i]]))
    if t >  3 / f:
        break
    
q_record = np.vstack(q_record)
q_record_ref = np.vstack(q_record_ref)


In [ ]:
import matplotlib.pyplot as plt
fig_width = 5
fig_height = 5
fig = plt.figure(figsize=(fig_width, fig_height))

ax = fig.add_subplot(211,)


line1,  = ax.plot(q_record[:,0], q_record[:,1], c='b',label='Actual')
line2,  = ax.plot(q_record_ref[:,0], q_record_ref[:,1], c='r', label='Ref')
ax.set_ylabel('q (rad)')

ax.legend(handles=[line1, line2], loc='upper right')

ax.set_xlabel('t (s)')
ax.grid(True)
ax.set_xlim([0, np.max(q_record[:,0])])
# fig.savefig('data_records/controller_test/vel_0' + str(f)+ ' .png',format='png',bbox_inches='tight', pad_inches=0.0, dpi=300)


# ax = fig.add_subplot(212,)
# 
# ax.plot(q_record[:,0], q_record[:,2])
# ax.plot(q_record_ref[:,0], q_record_ref[:,2])
# 
# ax.set_ylabel('$\dot{q}$ (rad/s)')

In [ ]:
hand_real.move_to_joints_by_vel(q_cmd)

In [ ]:
xy_step

In [ ]:
v[:2] * T/2 * 1 

In [4]:
## gait design for four legs
 # this is the init position

import copy
t0 = time.time()
r.f = 2
T = 1 / r.f
v = np.array([0.04, 0.02,0]) 
# v = np.array([0, -0.02,0])
h = 0.03  # z axis lifting height for swing legs
r.vel_cmd[:3] = v

xy_step = v[:2] * T/2 * 0.5  # based the MIT paper
x_cmd = copy.deepcopy(xh_local_init)
x_cmd = [a[:3] for a in x_cmd]  # position command for legs
q_cmd_list = []
t_total = 40 * T

k = 0
while 1:
    
    tau_Grf = np.zeros(20)
    t_now  = time.time() - t0
    t_period = t_now % (1/r.f)  # [0, T]
    
    if t_period < T/2: # FL and BR, front left and back right legs move during [0, T/2]
        swing_legs = [0, 3]
        stance_legs = [1, 4]
        # swing_legs = [0, 1]
        # stance_legs = [3, 4]
    else:
        swing_legs = [1, 4]
        stance_legs = [0, 3]
        # swing_legs = [3, 4]
        # stance_legs = [0, 1]
    
    # update vel direction in hand base frame
    # xy_step = rot.quat2mat(r.x[3:]).T @ np.array([xy_step[0], xy_step[1], 0])
    # xy_step = xy_step[:2]
    for i in swing_legs:
        t_tmp = t_period % (T/2)
        x_cmd[i][:2] = r.bezier(t_tmp, xh_local_init[i][:2] - xy_step, xh_local_init[i][:2] + xy_step)
        if t_tmp <= T/4:
            x_cmd[i][2] = r.bezier(t_tmp /(T/4), xh_local_init[i][2], xh_local_init[i][2] + h)  # z axis
        else:
            x_cmd[i][2] = r.bezier((t_tmp - T/4) /(T/4), xh_local_init[i][2] + h, xh_local_init[i][2])
    
    for i in stance_legs:
        t_tmp = t_period % (T/2)
        x_cmd[i][:2] = r.bezier(t_tmp /(T/2), xh_local_init[i][:2] + xy_step, xh_local_init[i][:2] - xy_step)
        x_cmd[i][2] = xh_local_init[i][2]
    
    q_desired, tau = r.Cartesian_space_cmd(x_cmd, return_q=True, return_tau=True)  # (20,)
    leg_force_opt = r.Grf_ref_opt()
    for i in range(4):
        leg_index = r.used_fingers[i]
        #                (4, 3)                         (3,3)                 (3,)
        tau_Grf[leg_index*4:leg_index*4+4] = - r.jac_local[leg_index].T @ rot.quat2mat(r.x[3:]).T @ leg_force_opt[i * 3: i * 3 + 3, 0]  # (4,)
    i = 2
    tau_Grf = np.zeros(20)
    tau = tau + tau_Grf  # (20,)
    tau[i*4: i*4+4] =  r.C[i*4: i*4+4] + r.M[i*4:i*4+4, i*4:i*4+4] @ ( 40000 * (q0[i*4:i*4+4] - r.q[i*4:i*4+4]) + 200 * (np.zeros(4) - r.dq[i*4:i*4+4]))
    r.send_torque(tau, torque_limit=1)
    
    # k += 1
    # q_cmd = q_sim2real(q_desired)
    # q_cmd_list.append(q_cmd)
    # hand_real.send_position(q_cmd)

    if t_now > t_total:
        break
    

In [ ]:
q_cmd_list = np.vstack(q_cmd_list)
q_cmd_list.shape

In [ ]:
hand_real.move_to_joints(q_cmd_list, t=t_total)

In [ ]:
q_cmd = q_sim2real(q0)
hand_real.move_to_joints(q_cmd, sim2real=False)  # move to init pose

### qp formulation, 
$$$\frac{1}{2} \boldsymbol{x}^T H \boldsymbol{x} +  C^T \boldsymbol{x} \\
s.t.    A \boldsymbol{x} = b \\
lb \leq \boldsymbol{x}\leq ub
$$$

In [ ]:
hand_real.move_to_joints(r.q,  t=2, sim2real=True)



In [ ]:
r.q.shape